In [289]:
%matplotlib inline
import math
import librosa


In [91]:
# see: https://www.daniweb.com/programming/software-development/code/216976/play-a-midi-music-file-using-pygame

# sudo pip install pygame

# on ubuntu
# sudo apt-get install python-pygame

import pygame

def play_music(music_file):
    """
    stream music with mixer.music module in blocking manner
    this will stream the sound from disk while playing
    """
    clock = pygame.time.Clock()
    try:
        pygame.mixer.music.load(music_file)
        print("Music file %s loaded!" % music_file)
    except pygame.error:
        print("File %s not found! (%s)" % (music_file, pygame.get_error()))
        return
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        # check if playback has finished
        clock.tick(30)
# pick a midi music file you have ...
# (if not in working folder use full path)

midi_file = '/home/ds/dev/ext/dsnotebooks/muz/data/chpn_op25_e2.mid'
freq = 44100    # audio CD quality
bitsize = -16   # unsigned 16 bit
channels = 2    # 1 is mono, 2 is stereo
buffer = 1024    # number of samples
pygame.mixer.init(freq, bitsize, channels, buffer)

# optional volume 0 to 1.0
pygame.mixer.music.set_volume(0.8)
try:
    play_music(midi_file)
except KeyboardInterrupt:
    # if user hits Ctrl/C then exit
    # (works only in console mode)
    pygame.mixer.music.fadeout(1000)
    pygame.mixer.music.stop()
    raise SystemExit

error: No available audio device

In [151]:
import numpy as np
import librosa as lr
from librosa import display
from mido import MidiFile



def to_piano_roll(midi):
    """Convert MIDI file to a 2D NumPy ndarray (notes, timesteps)."""
    notes = 127
    tempo = 500000  # Assume same default tempo and 4/4 for all MIDI files.
    seconds_per_beat = tempo / 1000000.0
    seconds_per_tick = seconds_per_beat / midi.ticks_per_beat
    velocities = np.zeros(notes)
    sequence = []
    for m in midi:
        ticks = int(np.round(m.time / seconds_per_tick))
        ls = [velocities.copy()] * ticks
        sequence.extend(ls)
        if m.type == 'note_on':
            velocities[m.note] = m.velocity
        elif m.type == 'note_off':
            velocities[m.note] = 0
        else:
            continue
    piano_roll = np.array(sequence).T
    return piano_roll



In [262]:
velocities = np.zeros(11*12)
velocities.reshape((11,12))

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [258]:
n = 106

def to_table_coord(n, num_col=12):
    return int(n/num_col), n % num_col
def from_table_coord(row, col, num_col=12):
    return row*num_col+col%num_col

z=to_table_coord(n)
print(z)
from_table_coord(z[0],z[1])


(8, 10)


106

In [273]:
import numpy as np
import librosa as lr
from librosa import display
from mido import MidiFile



def to_piano_roll(midi):
    """Convert MIDI file to a 2D NumPy ndarray (notes, timesteps)."""
    octaves=11
    notes = 12
    flat_notes = octaves*notes #127
    tempo = 500000  # Assume same default tempo and 4/4 for all MIDI files.
    seconds_per_beat = tempo / 1000000.0
    seconds_per_tick = seconds_per_beat / midi.ticks_per_beat
    velocities = np.zeros(flat_notes).reshape((octaves,notes))    
    sequence = []
    for m in list(midi)[0:100]:
        print(m)
        ticks = int(np.round(m.time / seconds_per_tick))
        ls = [velocities.copy()] * ticks
        sequence.extend(ls)
        print("len(sequence)",len(sequence))
        if m.type == 'note_on':
            velocities[to_table_coord(m.note)] = m.velocity
            print(velocities)
        elif m.type == 'note_off':
            velocities[to_table_coord(m.note)] = 0
            print(velocities)
        else:
            continue
    piano_roll = np.array(sequence).T
    return piano_roll



In [291]:
y, sr = librosa.load(librosa.util.example_audio_file())
y

NoBackendError: 

In [280]:
lr.get_duration()

ParameterError: Improper note format: C,D

In [274]:
f1 = MidiFile(midi_file)
piano_roll = to_piano_roll(f1)
piano_roll.shape

<meta message track_name name='Chopin Etude  No. 2, Opus 25' time=0>
len(sequence) 0
<meta message copyright text='Copyright © 2003 by Bernd Krueger ' time=0>
len(sequence) 0
<meta message text text='Frederic Chopin' time=0>
len(sequence) 0
<meta message text text='Presto' time=0>
len(sequence) 0
<meta message text text='Fertiggestellt am 31.1.2003\n' time=0>
len(sequence) 0
<meta message text text='Update am 16.2.2014\n' time=0>
len(sequence) 0
<meta message text text='Dauer: 1:31 Minuten\n' time=0>
len(sequence) 0
<meta message smpte_offset frame_rate=25 hours=32 minutes=0 seconds=3 frames=0 sub_frames=0 time=0>
len(sequence) 0
<meta message time_signature numerator=2 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
len(sequence) 0
<meta message key_signature key='Ab' time=0>
len(sequence) 0
<meta message set_tempo tempo=572355 time=0>
len(sequence) 0
<meta message track_name name='Piano right' time=0>
len(sequence) 0
program_change channel=0 program=0 time=0
l

(12, 11, 2637)

116473

In [268]:
display.specshow(piano_roll);

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [130]:
notes_mess = list(filter(lambda m: m.any(), piano_roll.T))

In [131]:
m = notes_mess[0]
m.argmax()

72

In [135]:
print(notes_mess[0])

[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  51.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.]


In [141]:
i = 0
t = 0
for m in notes_mess:
    print(m.argmax(),m.max())

72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0
72 51.0


In [97]:
f1.tracks

[<midi track 'Chopin Etude  No. 2, Opus 25' 797 messages>,
 <midi track 'Piano right' 1633 messages>,
 <midi track 'Piano left' 852 messages>,
 <midi track 'Chopin: Etude No.2 in F Minor Op. 25' 2 messages>,
 <midi track 'Copyright © 2003 by Bernd Krueger' 2 messages>,
 <midi track 'http://www.piano-midi.de' 2 messages>,
 <midi track 'Edition: 2014-02-16' 2 messages>,
 <midi track 'Spur 7' 2 messages>]

In [150]:
for m in list(f1)[0:100]:
    print(m)


<meta message track_name name='Chopin Etude  No. 2, Opus 25' time=0>
<meta message copyright text='Copyright © 2003 by Bernd Krueger ' time=0>
<meta message text text='Frederic Chopin' time=0>
<meta message text text='Presto' time=0>
<meta message text text='Fertiggestellt am 31.1.2003\n' time=0>
<meta message text text='Update am 16.2.2014\n' time=0>
<meta message text text='Dauer: 1:31 Minuten\n' time=0>
<meta message smpte_offset frame_rate=25 hours=32 minutes=0 seconds=3 frames=0 sub_frames=0 time=0>
<meta message time_signature numerator=2 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='Ab' time=0>
<meta message set_tempo tempo=572355 time=0>
<meta message track_name name='Piano right' time=0>
program_change channel=0 program=0 time=0
control_change channel=0 control=7 value=100 time=0
control_change channel=0 control=10 value=64 time=0
<meta message text text='bdca426d104a26ac9dcb070447587523' time=0>
control_change channel

In [108]:
len(list(f1))


3285